In [1]:
import math
import numpy as np
import tensorflow as tf
import tensorflow.keras
import tensorflow.keras.backend as K
import tensorflow.keras.losses
import tensorflow.keras.metrics
import tensorflow.keras.optimizers
from tensorflow.keras.layers import (
    BatchNormalization,
    Concatenate,
    Conv2D,
    Dense,
    Dropout,
    Flatten,
    GlobalAveragePooling1D,
    GlobalAveragePooling2D,
    Input,
    LayerNormalization,
    Lambda,
    LeakyReLU,
    MaxPooling2D,
    ReLU,
    Reshape,
)
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from config import input_shape, input_shape_mf
from CBAM3D import CBAM
from cvt_custom_layers import Transformer, LastStage, RearrangeLayer

2025-06-04 00:35:32.010549: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-04 00:35:32.070143: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from tensorflow.keras import mixed_precision

policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

In [3]:
K.set_image_data_format("channels_last")

In [4]:
model_name = "CVT_ortles_nu_flavour_evtau"

In [5]:
batch_size = 256 # Batch size
num_classes = 2 # Number of classes. 1 for regression, 2 for binary classification, n for multiclass classification
initial_dim = 16 #  Initial number of filters/channels in the network
kernels = [(3,9), (3,6), 3] # Square kernel dimensions for convolution layers
strides = [(4,4), (2,4), 2] # Stride values for convolutional layers
heads = [1, 3, 6] # Number of heads for the attention mechanism
depth = [1, 2, 5] # Refers to the number of layers
dropout = 0.1 # Transformer dropout value
scale_dim = 2 # Scaling factor for dim parameter over the 3 layers of cvt

In [6]:
### X_h STAGE 1 ###
dim = initial_dim
target_h_input = Input(input_shape, name="target_h_in")
X_h = Conv2D(filters=dim, kernel_size=kernels[0], strides=strides[0], padding='same')(target_h_input)
l, w = K.int_shape(X_h)[1], K.int_shape(X_h)[2]
#X_h = CBAM()(X_h)
X_h = RearrangeLayer(dim=dim, length=math.ceil(l), width=math.ceil(w), compression=True)(X_h)
X_h = LayerNormalization()(X_h)
X_h = Transformer(dim=dim, length=math.ceil(l), width=math.ceil(w), depth=depth[0], heads=heads[0], dim_head=dim,
                                            mlp_dim=dim*scale_dim, dropout=dropout)(X_h, training=True)
X_h = RearrangeLayer(dim=dim, length=math.ceil(l), width=math.ceil(w))(X_h)

### X_h STAGE 2 ###
scale = heads[1]//heads[0]
dim = scale*dim
X_h = Conv2D(filters=dim, kernel_size=kernels[1], strides=strides[1], padding='same')(X_h)
l, w = K.int_shape(X_h)[1], K.int_shape(X_h)[2]
#X_h = CBAM()(X_h)
X_h = RearrangeLayer(dim=dim, length=math.ceil(l), width=math.ceil(w), compression=True)(X_h)
X_h = LayerNormalization()(X_h)
X_h = Transformer(dim=dim, length=math.ceil(l), width=math.ceil(w), depth=depth[1], heads=heads[1], dim_head=dim,
                                            mlp_dim=dim*scale_dim, dropout=dropout)(X_h, training=True)
X_h = RearrangeLayer(dim=dim, length=math.ceil(l), width=math.ceil(w))(X_h)

### X_h STAGE 3 ###
scale = heads[2] // heads[1]
dim = scale*dim
X_h = Conv2D(filters=dim, kernel_size=kernels[2], strides=strides[2], padding='same')(X_h)
l, w = K.int_shape(X_h)[1], K.int_shape(X_h)[2]
#X_h = CBAM()(X_h)
X_h = RearrangeLayer(dim=dim, length=math.ceil(l), width=math.ceil(w), compression=True)(X_h)
X_h = LayerNormalization()(X_h)
X_h = LastStage(dim=dim, batch_size=batch_size)(X_h, training=True)
X_h = Transformer(dim=dim, length=math.ceil(l), width=math.ceil(w), depth=depth[2], heads=heads[2], dim_head=dim,
                                            mlp_dim=dim*scale_dim, dropout=dropout, last_stage=True)(X_h, training=True)
if num_classes == 1:
    X_h = GlobalAveragePooling1D()(X_h)
else:
    X_h = X_h[:, 0]
    
X_h = LayerNormalization()(X_h)
X_h = Dense(dim)(X_h)
X_h = Flatten()(X_h)


### X_v STAGE 1 ###
dim = initial_dim
target_v_input = Input(input_shape, name="target_v_in")
X_v = Conv2D(filters=dim, kernel_size=kernels[0], strides=strides[0], padding='same')(target_v_input)
l, w = K.int_shape(X_v)[1], K.int_shape(X_v)[2]
#X_v = CBAM()(X_v)
X_v = RearrangeLayer(dim=dim, length=math.ceil(l), width=math.ceil(w), compression=True)(X_v)
X_v = LayerNormalization()(X_v)
X_v = Transformer(dim=dim, length=math.ceil(l), width=math.ceil(w), depth=depth[0], heads=heads[0], dim_head=dim,
                                            mlp_dim=dim*scale_dim, dropout=dropout)(X_v, training=True)
X_v = RearrangeLayer(dim=dim, length=math.ceil(l), width=math.ceil(w))(X_v)

### X_v STAGE 2 ###
scale = heads[1]//heads[0]
dim = scale*dim
X_v = Conv2D(filters=dim, kernel_size=kernels[1], strides=strides[1], padding='same')(X_v)
l, w = K.int_shape(X_v)[1], K.int_shape(X_v)[2]
#X_v = CBAM()(X_v)
X_v = RearrangeLayer(dim=dim, length=math.ceil(l), width=math.ceil(w), compression=True)(X_v)
X_v = LayerNormalization()(X_v)
X_v = Transformer(dim=dim, length=math.ceil(l), width=math.ceil(w), depth=depth[1], heads=heads[1], dim_head=dim,
                                            mlp_dim=dim*scale_dim, dropout=dropout)(X_v, training=True)
X_v = RearrangeLayer(dim=dim, length=math.ceil(l), width=math.ceil(w))(X_v)

### X_v STAGE 3 ###
scale = heads[2] // heads[1]
dim = scale*dim
X_v = Conv2D(filters=dim, kernel_size=kernels[2], strides=strides[2], padding='same')(X_v)
l, w = K.int_shape(X_v)[1], K.int_shape(X_v)[2]
#X_v = CBAM()(X_v)
X_v = RearrangeLayer(dim=dim, length=math.ceil(l), width=math.ceil(w), compression=True)(X_v)
X_v = LayerNormalization()(X_v)
X_v = LastStage(dim=dim, batch_size=batch_size)(X_v, training=True)
X_v = Transformer(dim=dim, length=math.ceil(l), width=math.ceil(w), depth=depth[2], heads=heads[2], dim_head=dim,
                                            mlp_dim=dim*scale_dim, dropout=dropout, last_stage=True)(X_v, training=True)
if num_classes == 1:
    X_v = GlobalAveragePooling1D()(X_v)
else:
    X_v = X_v[:, 0]
    
X_v = LayerNormalization()(X_v)
X_v = Dense(dim)(X_v)
X_v = Flatten()(X_v)


### X_mf_h STAGE 1 ###
dim = initial_dim
mufilter_in_h = Input(input_shape_mf, name="mufilter_in_h")
X_mf_h = Conv2D(filters=dim, kernel_size=kernels[0], strides=strides[0], padding='same')(mufilter_in_h)
l, w = K.int_shape(X_mf_h)[1], K.int_shape(X_mf_h)[2]
#X_mf_h = CBAM()(X_mf_h)
X_mf_h = RearrangeLayer(dim=dim, length=math.ceil(l), width=math.ceil(w), compression=True)(X_mf_h)
X_mf_h = LayerNormalization()(X_mf_h)
X_mf_h = Transformer(dim=dim, length=math.ceil(l), width=math.ceil(w), depth=depth[0], heads=heads[0], dim_head=dim,
                                            mlp_dim=dim*scale_dim, dropout=dropout)(X_mf_h, training=True)
X_mf_h = RearrangeLayer(dim=dim, length=math.ceil(l), width=math.ceil(w))(X_mf_h)

### X_mf_h STAGE 2 ###
scale = heads[1] // heads[0]
dim = scale * dim
X_mf_h = Conv2D(filters=dim, kernel_size=kernels[1], strides=strides[1], padding='same')(X_mf_h)
l, w = K.int_shape(X_mf_h)[1], K.int_shape(X_mf_h)[2]
#X_mf_h = CBAM()(X_mf_h)
X_mf_h = RearrangeLayer(dim=dim, length=math.ceil(l), width=math.ceil(w), compression=True)(X_mf_h)
X_mf_h = LayerNormalization()(X_mf_h)
X_mf_h = Transformer(dim=dim, length=math.ceil(l), width=math.ceil(w), depth=depth[1], heads=heads[1], dim_head=dim,
                                            mlp_dim=dim*scale_dim, dropout=dropout)(X_mf_h, training=True)
X_mf_h = RearrangeLayer(dim=dim, length=math.ceil(l), width=math.ceil(w))(X_mf_h)

### X_mf_h STAGE 3 ###
scale = heads[2] // heads[1]
dim = scale * dim
X_mf_h = Conv2D(filters=dim, kernel_size=kernels[2], strides=strides[2], padding='same')(X_mf_h)
l, w = K.int_shape(X_mf_h)[1], K.int_shape(X_mf_h)[2]
#X_mf_h = CBAM()(X_mf_h)
X_mf_h = RearrangeLayer(dim=dim, length=math.ceil(l), width=math.ceil(w), compression=True)(X_mf_h)
X_mf_h = LayerNormalization()(X_mf_h)
X_mf_h = LastStage(dim=dim, batch_size=batch_size)(X_mf_h, training=True)
X_mf_h = Transformer(dim=dim, length=math.ceil(l), width=math.ceil(w), depth=depth[2], heads=heads[2], dim_head=dim,
                                            mlp_dim=dim*scale_dim, dropout=dropout, last_stage=True)(X_mf_h, training=True)
if num_classes == 1:
    X_mf_h = GlobalAveragePooling1D()(X_mf_h)
else:
    X_mf_h = X_mf_h[:, 0]
    
X_mf_h = LayerNormalization()(X_mf_h)
X_mf_h = Dense(dim)(X_mf_h)
X_mf_h = Flatten()(X_mf_h)


### X_mf_v STAGE 1 ###
dim = initial_dim
mufilter_in_v = Input(input_shape_mf, name="mufilter_in_v")
X_mf_v = Conv2D(filters=dim, kernel_size=kernels[0], strides=strides[0], padding='same')(mufilter_in_v)
l, w = K.int_shape(X_mf_v)[1], K.int_shape(X_mf_v)[2]
#X_mf_v = CBAM()(X_mf_v)
X_mf_v = RearrangeLayer(dim=dim, length=math.ceil(l), width=math.ceil(w), compression=True)(X_mf_v)
X_mf_v = LayerNormalization()(X_mf_v)
X_mf_v = Transformer(dim=dim, length=math.ceil(l), width=math.ceil(w), depth=depth[0], heads=heads[0], dim_head=dim,
                                            mlp_dim=dim*scale_dim, dropout=dropout)(X_mf_v, training=True)
X_mf_v = RearrangeLayer(dim=dim, length=math.ceil(l), width=math.ceil(w))(X_mf_v)

### X_mf_v STAGE 2 ###
scale = heads[1] // heads[0]
dim = scale * dim
X_mf_v = Conv2D(filters=dim, kernel_size=kernels[1], strides=strides[1], padding='same')(X_mf_v)
l, w = K.int_shape(X_mf_v)[1], K.int_shape(X_mf_v)[2]
#X_mf_v = CBAM()(X_mf_v)
X_mf_v = RearrangeLayer(dim=dim, length=math.ceil(l), width=math.ceil(w), compression=True)(X_mf_v)
X_mf_v = LayerNormalization()(X_mf_v)
X_mf_v = Transformer(dim=dim, length=math.ceil(l), width=math.ceil(w), depth=depth[1], heads=heads[1], dim_head=dim,
                                            mlp_dim=dim*scale_dim, dropout=dropout)(X_mf_v, training=True)
X_mf_v = RearrangeLayer(dim=dim, length=math.ceil(l), width=math.ceil(w))(X_mf_v)

### X_mf_v STAGE 3 ###
scale = heads[2] // heads[1]
dim = scale * dim
X_mf_v = Conv2D(filters=dim, kernel_size=kernels[2], strides=strides[2], padding='same')(X_mf_v)
l, w = K.int_shape(X_mf_v)[1], K.int_shape(X_mf_v)[2]
#X_mf_v = CBAM()(X_mf_v)
X_mf_v = RearrangeLayer(dim=dim, length=math.ceil(l), width=math.ceil(w), compression=True)(X_mf_v)
X_mf_v = LayerNormalization()(X_mf_v)
X_mf_v = LastStage(dim=dim, batch_size=batch_size)(X_mf_v, training=True)
X_mf_v = Transformer(dim=dim, length=math.ceil(l), width=math.ceil(w), depth=depth[2], heads=heads[2], dim_head=dim,
                                            mlp_dim=dim*scale_dim, dropout=dropout, last_stage=True)(X_mf_v, training=True)
if num_classes == 1:
    X_mf_v = GlobalAveragePooling1D()(X_mf_v)
else:
    X_mf_v = X_mf_v[:, 0]
    
X_mf_v = LayerNormalization()(X_mf_v)
X_mf_v = Dense(dim)(X_mf_v)
X_mf_v = Flatten()(X_mf_v)



X = Concatenate()([X_h, X_v, X_mf_h, X_mf_v])
X = Dense(128)(X)
X = BatchNormalization()(X)
X = LeakyReLU(negative_slope=0.1)(X)
X = Dropout(0.2)(X)
X = Dense(64)(X)
X = BatchNormalization()(X)
X = LeakyReLU(negative_slope=0.1)(X)
X = Dropout(0.2)(X)

if num_classes == 1:
    X = Dense(1)(X)
else:
    activation = 'sigmoid' if num_classes == 2 else 'softmax'
    units = 1 if num_classes == 2 else num_classes
    X = Dense(units, activation=activation)(X)
        
model = Model(
    inputs=[target_h_input, target_v_input, mufilter_in_h, mufilter_in_v],
    outputs=X,
    name=model_name,
)

K.clear_session()

if num_classes == 1:
    model.compile(
    optimizer="Adam",
    loss="mse",
    metrics=[
        "mae",
    ],
)
else:
    if num_classes == 2:
        model.compile(
            optimizer="Adam",
            loss=tensorflow.keras.losses.BinaryFocalCrossentropy(from_logits=True),
            metrics=[
                tensorflow.keras.metrics.BinaryAccuracy(name="acc", threshold=0.5),
                tensorflow.keras.metrics.AUC(from_logits=True),
            ],
        )
    else:
        model.compile(
            optimizer="Adam",
            loss=tensorflow.keras.losses.CategoricalCrossentropy(from_logits=True),
            metrics=[
                tensorflow.keras.metrics.CategoricalAccuracy(name="acc"),
                tensorflow.keras.metrics.AUC(from_logits=True),
            ],
        )

2025-06-04 00:35:39.698305: E tensorflow/core/util/util.cc:131] oneDNN supports DT_HALF only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.


In [7]:
model.save(f"model_zoo/{model_name}_b{batch_size}_e0.keras")

In [8]:
model.summary()
plot_model(
    model,
    to_file= (f"model_zoo/{model_name}_b{batch_size}_e0.png"),
    show_shapes=True,
    show_dtype=True,
    show_layer_names=True,
    rankdir="TB",  # Top to bottom (TB) or left to right (LR)
    expand_nested=True
)

Model: "CVT_ortles_nu_flavour_evtau"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ target_h_in         │ (None, 58, 3279,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ target_v_in         │ (None, 58, 3279,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mufilter_in_h       │ (None, 34, 3279,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mufilter_in_v       │ (None, 34, 3279,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cast (Cast)         │ (None, 58, 3279,  │          0 │ target_h_in[0][0] │
│                     │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cast_1 (Cast)       │ (None, 58, 3279,  │          0 │ target_v_in[0][0] │
│                     │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cast_2 (Cast)       │ (None, 34, 3279,  │          0 │ mufilter_in_h[0]… │
│                     │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cast_3 (Cast)       │ (None, 34, 3279,  │          0 │ mufilter_in_v[0]… │
│                     │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 15, 820,   │        448 │ cast[0][0]        │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_27 (Conv2D)  │ (None, 15, 820,   │        448 │ cast_1[0][0]      │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_54 (Conv2D)  │ (None, 9, 820,    │        448 │ cast_2[0][0]      │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_81 (Conv2D)  │ (None, 9, 820,    │        448 │ cast_3[0][0]      │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rearrange_layer     │ (None, 12300, 16) │          0 │ conv2d[0][0]      │
│ (RearrangeLayer)    │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rearrange_layer_5   │ (None, 12300, 16) │          0 │ conv2d_27[0][0]   │
│ (RearrangeLayer)    │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rearrange_layer_10  │ (None, 7380, 16)  │          0 │ conv2d_54[0][0]   │
│ (RearrangeLayer)    │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rearrange_layer_15  │ (None, 7380, 16)  │          0 │ conv2d_81[0][0]   │
│ (RearrangeLayer)    │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, 12300, 16) │         32 │ rearrange_layer[

 Total params: 2,075,841 (7.92 MB)

 Trainable params: 2,061,249 (7.86 MB)

 Non-trainable params: 14,592 (57.00 KB)